In [4]:
import calour as ca
import calour_utils as cu

failed to load logging config file


In [5]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import os
import pandas as pd
import shutil
import subprocess

In [6]:
ca.set_log_level('INFO')

In [7]:
%matplotlib inline

In [8]:
pwd

'/Users/amnon/git/paper-metaanalysis/scripts'

In [9]:
pd.__version__

'1.2.5'

In [10]:
from os.path import join
save_dir = '../lefse_ratios/'

# Prepare the ratios table
## For each study in the directory, calculate the sick/healthy ratio for each bacteria
### Steps:
#### For each study identify bacteria significantly different between sick/healthy
Using at most 23 sick and 10 healthy samples per study (for fair comparison between study sizes)

Using FDR=0.25
#### Join the significant bacteria from all studies
To a single list
#### Calculate the log2(sick/healthy) for each bacteria in each study
On all samples from each study (to get better estimate of the effect size).

Save in a biom table. 1 entry per bacteria per study (log2 ratio sick/healthy)

In [10]:
def save_for_lefse(exp, field, out_file):
    '''Save a calour experiment for Lefse analysis (adding sample id in first row, field values the 2nd row)
    After calling this function, run:
        lefse_format_input.py OUT_FILE PREPARED_FILE_NAME -c 2 -u 1
    and then:
        lefse_run.py PREPARED_FILE_NAME LEFSE_OUTPUT_FILE -a 1 -l 2 -w 1
    '''
    df = exp.to_pandas(sparse=False).transpose()
    header_field=exp.sample_metadata[field].to_frame().transpose()
    df = header_field.append(df)
    df.to_csv(out_file,sep='\t')

In [11]:
def norm_diff_lefse(exp, outfile, field='type', val1='HC', val2='disease', alpha=0.25, **kwargs):
    '''identify significant bacteria and effect sizes using lefse
    
    Parameters
    ----------
    
    Returns
    -------
    calour.AmpliconExperiment with significant (level alpha no FDR) features differentiating between val1 and val2 in field
    _calour_stat is negative for val2, positive for val1
    '''
    import subprocess

    exp=exp.filter_samples(field,[val1, val2])
    save_for_lefse(exp, field, outfile+'.table.txt')
    res=subprocess.run(['lefse_format_input.py',outfile+'.table.txt', outfile+'.lefse.prep','-c','2','-u','1'],stderr=subprocess.PIPE, text=True)    
    if res.returncode!=0:
        print('error on lefse_format_input.py for file %s:\n%s' % (outfile, res.stderr))
        return None

    res=subprocess.run(['lefse_run.py',outfile+'.lefse.prep',outfile+'.lefse.txt','-a','1','-l','0','-w','1'], stderr=subprocess.PIPE, text=True)
    if res.returncode!=0:
        print('error on lefse_run.py for file %s:\n%s' % (outfile, res.stderr))
        return None
    
    df=pd.read_csv(outfile+'.lefse.txt',sep='\t',names=['seq','wilcox','group','lda','pval'],index_col=0)
    df['pval']=df.pval.replace('-',2).astype(float)

    # filter keeping only pval <= alpha
    df=df[df['pval']<=alpha]
    
    df['_calour_stat']=df['lda'].copy()
    # fix the direction of the lda to negative for higher in val2
    df.loc[df['group']==val2,'_calour_stat']=-1*df.loc[df['group']==val2,'_calour_stat']
    df['_calour_direction']=df['group'].copy()
    df['_calour_pval']=df['pval'].copy()
    df['_feature_id']=df.index.values

    # create the experiment
    ca.set_log_level('INFO')
    dd=exp.filter_ids(df.index.values)
    dd.feature_metadata=df

    dd=dd.sort_samples(field)
    dd=dd.sort_by_metadata('_calour_stat',axis='f')
    return dd

In [12]:
def diff_all(fasta_file=None, alpha=0.25, subset_size=None, subset_control=None, random_seed=None):
    '''Iterate over "studies" directory. Each subdir is an experiment with a biom table (all.XXX.biom) and map file (up.map.csv).
    The map file contains the field "type" with the values "HC"/"disease".
    If fasta_file is not None, we keep only bacteria from the experiment present in fasta_file (for second stage diff abundace to reduce FDR cost)
    We then do diff. abundance with thresh. alpha
    Finally we save these disease associated bacteria to a new biom table diff-ALPHA.biom
    
    Parameters
    ----------
    fasta_file:str or None, optional
        if not None, keep only bacteria present in fasta_file
    alpha: float, optional
        the alpha value for the dsFDR (see diff_abundance()  )
    subset_size: int or None, optional
        if not None, keep only subset_size samples from each group (HC/disease)
    subset_control: int or None, optional
        if None,  control subset_size is the same as subset_size.
        if not None, this is the minimal control subset_size
    '''
    x = []
    savedir = join(save_dir,'diff-%s' % alpha)
    if subset_control is None:
        subset_control = subset_size
    if subset_size is not None:
        savedir += '-subset-%d' % subset_size
    print('deleting output dir %s' % savedir)
    try:
        shutil.rmtree(savedir)
    except:
        pass
    if fasta_file is not None:
        savedir += '-filtered'
    try:
        os.mkdir(savedir)
    except:
        pass
    num_processed = 0
    for cname in glob.glob('../studies/*'):
        if os.path.isdir(cname):
            # re-set the random seed for each directory since order of directories may vary between OSs
            if random_seed is not None:
                np.random.seed(random_seed)
            print('**********')
            print(cname)
            x.append(cname)
            tables = glob.glob(os.path.join(cname,'all.*biom'))
            print(tables)
            if len(tables)==0:
                print('dir %s does not contain a biom table' % cname)
                continue
            bt=tables[0]
            data=ca.read_amplicon(os.path.join(bt),os.path.join(cname,'up.map.csv'),normalize=10000,min_reads=1000,sparse=False)
            print('-------------')
            print(data)
            # if enough samples, keep only random subet
            if subset_size is not None:
                tt=data.filter_samples('type','HC')
                ns = len(tt.sample_metadata)
                if ns < subset_control:
                    print('not enough controls (%d). skipping' % ns)
                    continue
                # randomly subsample
                rp = np.random.permutation(ns)
                rp = rp[:subset_size]
                new_ids = [tt.sample_metadata['_sample_id'][x] for x in rp]
                # similar for disease
                tt=data.filter_samples('type','disease')
                ns = len(tt.sample_metadata)
                if ns < subset_size:
                    print('not enough disease (%d). skipping' % ns)
                    continue
                # randomly subsample
                rp = np.random.permutation(ns)
                rp = rp[:subset_size]
                new_ids2 = [tt.sample_metadata['_sample_id'][x] for x in rp]
                # join the ids for the sick and controls
                all_ids = new_ids + new_ids2
                # and finally
                data = data.filter_samples('_sample_id',all_ids)
                print('keeping')
            data=data.filter_sum_abundance(10)
            if fasta_file is not None:
                data=data.filter_by_fasta(fasta_file)
            # dd=norm_diff(data,'type','disease', 'HC',alpha=alpha,random_seed=1234)
            dd=norm_diff_lefse(data,os.path.join(cname,'lefse'),'type','disease', 'HC',alpha=alpha)
            savename = os.path.join(savedir,'diff-%s' % (os.path.basename(cname)))
            dd.save_biom(savename+'.biom',add_metadata=None)
            #dd.save_metadata(savename+'.tsv',axis='f')
            dd.save(savename)
            num_processed += 1
    print('processed %d studies' % num_processed)

In [13]:
ca.set_log_level('ERROR')

In [14]:
diff_all(alpha=0.1,subset_size=23, subset_control=10, random_seed=2020)

deleting output dir ../lefse_ratios/diff-0.1-subset-23
**********
../studies/61
['../studies/61/all.biom']
-------------
AmpliconExperiment with 41 samples, 2715 features
not enough disease (20). skipping
**********
../studies/59
['../studies/59/all.biom']
-------------
AmpliconExperiment with 33 samples, 2637 features
not enough disease (17). skipping
**********
../studies/50
['../studies/50/all.biom']
-------------
AmpliconExperiment with 58 samples, 959 features
keeping
Number of significantly discriminative features: 787 ( 787 ) before internal wilcoxon
Number of discriminative features with abs LDA score > 0.0 : 787
2022-01-01 23:56:20 INFO Metadata field taxonomy not found. Saving biom table without metadata
**********
../studies/57
['../studies/57/all.biom']
2022-01-01 23:56:21 INFO loaded 85 samples, 4058 features
2022-01-01 23:56:21 WARNING Do you forget to normalize your data? It is required before running this function
2022-01-01 23:56:21 INFO After filtering, 85 remain.
---

# load everything and save to a single table and fasta file

In [15]:
def merge_results(files_dir):
    '''Load the results of the diff_all() and merge to a single table
    save the sequences to "combined-FILES-DIR.fasta"
    returns the Experiment
    '''
    df = None
    all_filenames = glob.glob(join(save_dir, files_dir) + "/*_feature.txt")
    print(all_filenames)
    for j,f in enumerate(all_filenames):
        dt=pd.read_csv((f), sep='\t')
        # print(dt)
        if '_feature_id' not in dt.columns:
            dt['_feature_id']=dt['seq'].copy()
        dt=dt[['_feature_id', '_calour_stat']]
        ccolname='-'.join(os.path.basename(f).split('-')[1:]).split('_feature.txt')[0]
        dt=dt.rename(columns = { '_calour_stat': ccolname})
        if df is None:
            df = dt
        else:
            df=pd.merge(df, dt, how='outer', on = '_feature_id')
#     print('processed %d files' % (j+1))
    df.fillna(value=0, inplace=True)
    oname = join(save_dir, 'combined-%s' % files_dir)
    df.to_csv(oname + '.tsv', sep='\t', index=False, encoding='utf-8-sig')
    df=df.set_index('_feature_id',drop=False)
    exp = ca.Experiment.from_pandas(df.transpose())
    exp.save_fasta(oname+'.fasta')
    return exp

In [16]:
exp = merge_results('diff-0.25-subset-23')

['../lefse_ratios/diff-0.25-subset-23/diff-12_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-62_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-50_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-45_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-4_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-29_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-40_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-49_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-39_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-17_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-55_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-14_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-2_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-43_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-7_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-46_feature.txt', '../lefse_ratios/diff-0.25-subset-23/diff-18_feature.txt',

# redo diff abundance with fasta filtering
in order to get effect size. so we use alpha=1

In [17]:
diff_all(fasta_file=join(save_dir,'combined-diff-0.25-subset-23.fasta'),alpha=1,subset_size=None)

deleting output dir ../lefse_ratios/diff-1
**********
../studies/61
['../studies/61/all.biom']
2022-01-02 00:14:47 INFO loaded 41 samples, 2715 features
2022-01-02 00:14:47 WARNING Do you forget to normalize your data? It is required before running this function
2022-01-02 00:14:47 INFO After filtering, 41 remain.
-------------
AmpliconExperiment with 41 samples, 2715 features
2022-01-02 00:14:47 INFO After filtering, 892 remain.
Number of significantly discriminative features: 759 ( 759 ) before internal wilcoxon
Number of discriminative features with abs LDA score > 0.0 : 759
2022-01-02 00:15:11 INFO Metadata field taxonomy not found. Saving biom table without metadata
**********
../studies/59
['../studies/59/all.biom']
2022-01-02 00:15:11 INFO loaded 33 samples, 2637 features
2022-01-02 00:15:11 WARNING Do you forget to normalize your data? It is required before running this function
2022-01-02 00:15:11 INFO After filtering, 33 remain.
-------------
AmpliconExperiment with 33 sample

In [18]:
exp = merge_results('diff-1-filtered')

['../lefse_ratios/diff-1-filtered/diff-12_feature.txt', '../lefse_ratios/diff-1-filtered/diff-62_feature.txt', '../lefse_ratios/diff-1-filtered/diff-50_feature.txt', '../lefse_ratios/diff-1-filtered/diff-20_feature.txt', '../lefse_ratios/diff-1-filtered/diff-45_feature.txt', '../lefse_ratios/diff-1-filtered/diff-4_feature.txt', '../lefse_ratios/diff-1-filtered/diff-29_feature.txt', '../lefse_ratios/diff-1-filtered/diff-59_feature.txt', '../lefse_ratios/diff-1-filtered/diff-40_feature.txt', '../lefse_ratios/diff-1-filtered/diff-1_feature.txt', '../lefse_ratios/diff-1-filtered/diff-8_feature.txt', '../lefse_ratios/diff-1-filtered/diff-49_feature.txt', '../lefse_ratios/diff-1-filtered/diff-39_feature.txt', '../lefse_ratios/diff-1-filtered/diff-17_feature.txt', '../lefse_ratios/diff-1-filtered/diff-25_feature.txt', '../lefse_ratios/diff-1-filtered/diff-55_feature.txt', '../lefse_ratios/diff-1-filtered/diff-26_feature.txt', '../lefse_ratios/diff-1-filtered/diff-56_feature.txt', '../lefse_ra

In [19]:
def prepare_exp(f, o):
    '''Prepare the merged experiment for analysis and save it
    
    Parameters
    ----------
    f: str
        name of the tsv file output from merge_results() (XXX.tsv)
    o: str
        name of the output o.biom o.map.tsv
    
    Returns
    -------
    ca.Experiment
    '''
    data=pd.read_csv(join(save_dir,f),sep='\t',index_col='_feature_id')
    dat=data.to_numpy(dtype=float)
    newexp=ca.Experiment(dat.T,sample_metadata=pd.DataFrame(data.columns,index=data.columns),feature_metadata=pd.DataFrame(data.index, index=data.index))
    o = join(save_dir, o)
    newexp.save(o)
#    newexp.save_biom(o+'.biom',add_metadata=None)
#    newexp.save_metadata(o+'.map.tsv')
    xx=ca.read_amplicon(o+'.biom',o+'_sample.txt',normalize=None, min_reads=None)
    return xx

# Save the ratios table to 'ratios/ratios.biom'
## sample metadata in ratios_sample.txt

In [20]:
xx=prepare_exp('combined-diff-1-filtered.tsv','ratios')

2022-01-02 00:59:21 INFO Metadata field taxonomy not found. Saving biom table without metadata
2022-01-02 00:59:21 INFO loaded 59 samples, 2511 features


In [21]:
xx

AmpliconExperiment with 59 samples, 2511 features

In [22]:
xx.sparse=False

# Create effect size table for all features
Used to test the lefse effect direction (and mean) for all features identified as significant by NRMD

In [90]:
def effect_all():
    '''Create a single biom table for all effect size in all samples
    used for comparison to NRMD metric
    
    Returns
    -------
    pandas.dataframe with column for each study, row for each ASV, value is the LEFSE stat (positive is higher in HC)
    '''
    res=pd.DataFrame(columns=['Seq'])
    res=res.set_index('Seq')
    x = []
    savedir = join(save_dir,'effect_all')
    num_processed = 0
    for cname in glob.glob('../studies/*'):
        if os.path.isdir(cname):
            print('**********')
            print(cname)
            ccname = cname.split('/')[-1]
            x.append(cname)
            tables = glob.glob(os.path.join(cname,'lefse.lefse.txt'))
            print(tables)
            if len(tables)==0:
                print('dir %s does not contain a lefse output file' % cname)
                continue
            bt=tables[0]
            df=pd.read_csv(bt,sep='\t',names=['seq','wilcox','group','lda','pval'],index_col=0)
            val2='disease'
            df['_calour_stat']=df['lda'].copy()
            # fix the direction of the lda to negative for higher in val2
            df.loc[df['group']==val2,'lda']=-1*df.loc[df['group']==val2,'lda']
            df=df.loc[:,'lda']
            res=res.merge(df,left_index=True,right_index=True,how='outer')
            res.rename(columns={'lda':ccname}, inplace=True)
            print(len(res))
    return res

In [91]:
df=effect_all()

**********
../studies/61
['../studies/61/lefse.lefse.txt']
766
**********
../studies/59
['../studies/59/lefse.lefse.txt']
894
**********
../studies/50
['../studies/50/lefse.lefse.txt']
1111
**********
../studies/57
['../studies/57/lefse.lefse.txt']
1322
**********
../studies/32
['../studies/32/lefse.lefse.txt']
1462
**********
../studies/56
['../studies/56/lefse.lefse.txt']
1509
**********
../studies/51
['../studies/51/lefse.lefse.txt']
1711
**********
../studies/58
['../studies/58/lefse.lefse.txt']
1730
**********
../studies/60
['../studies/60/lefse.lefse.txt']
1771
**********
../studies/34
['../studies/34/lefse.lefse.txt']
1854
**********
../studies/33
['../studies/33/lefse.lefse.txt']
1881
**********
../studies/20
['../studies/20/lefse.lefse.txt']
1924
**********
../studies/18
['../studies/18/lefse.lefse.txt']
2002
**********
../studies/27
['../studies/27/lefse.lefse.txt']
2087
**********
../studies/9
['../studies/9/lefse.lefse.txt']
2185
**********
../studies/11
['../studies/11/lef

In [92]:
df.to_csv('../lefse_ratios/all_lefse_ratios.txt',sep='\t')